In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
else:
  print('Not running on CoLab')

Running on CoLab


In [4]:
# Install wandb and update it to the latest version
if 'google.colab' in str(get_ipython()):
    !pip install wandb --upgrade -q

     |████████████████████████████████| 1.9 MB 5.3 MB/s 
     |████████████████████████████████| 182 kB 47.6 MB/s 
     |████████████████████████████████| 166 kB 36.3 MB/s 
     |████████████████████████████████| 63 kB 914 kB/s 
     |████████████████████████████████| 166 kB 18.7 MB/s 
     |████████████████████████████████| 162 kB 65.7 MB/s 
     |████████████████████████████████| 162 kB 41.3 MB/s 
     |████████████████████████████████| 158 kB 30.4 MB/s 
     |████████████████████████████████| 157 kB 62.4 MB/s 
     |████████████████████████████████| 157 kB 40.5 MB/s 
     |████████████████████████████████| 157 kB 45.8 MB/s 
     |████████████████████████████████| 157 kB 52.7 MB/s 
     |████████████████████████████████| 157 kB 43.2 MB/s 
     |████████████████████████████████| 157 kB 44.0 MB/s 
     |████████████████████████████████| 157 kB 49.2 MB/s 
     |████████████████████████████████| 156 kB 24.8 MB/s 


In [5]:
# mount google drive
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Importing the necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext.vocab import  vocab

import random
from datetime import datetime
import numpy as np
import pandas as pd
import joblib
from collections import Counter
from types import SimpleNamespace

from pathlib import Path
import sys

from sklearn.model_selection import train_test_split
import wandb

In [8]:
base_folder = Path('/content/drive/MyDrive/NLP_Fall22/HW5')
data_folder = base_folder/'Data'
model_folder = base_folder/'Models'
custom_functions = base_folder/'Scripts/custom_function'

In [9]:
sys.path.append(str(custom_functions))

In [10]:
sys.path

['/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/MyDrive/NLP_Fall22/HW5/Scripts/custom_function']

In [11]:
# Login to W&B
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [12]:
import custom_preprocessor as cp

In [13]:
df = pd.read_csv(data_folder/'multiclass_hw_cleaned.csv')

In [14]:
df.head()

,Unnamed: 0,Title,Body,cleaned_text,Tags,Tag_Number_final
0,0,detail disclosure indicator on UIButton,<p>Is there a simple way to place a detail dis...,detail disclosure indicator uibutton simple wa...,iphone,8
1,1,hello world fails to show up in emulator,<p>I followed Hello World tutorial exactly. E...,hello world fail emulator follow hello world t...,android,4
2,2,"Why is JSHint throwing a ""possible strict viol...",<p>Trying to validate some Javascript in JsHin...,jshint throw possible strict violation line tr...,javascript,3
3,3,Programmatically Make Bound Column Invisible,<p>I'm trying to make a data bound column invi...,programmatically bound column invisible try da...,asp.net,9
4,4,"More than one EditText - not getting focus, no...",<p>The home screen of my Android application h...,edittext get focus soft keyboard android home ...,android,4


In [16]:
df = df[['cleaned_text', 'Tag_Number_final']]

In [17]:
df.head()

,cleaned_text,Tag_Number_final
0,detail disclosure indicator uibutton simple wa...,8
1,hello world fail emulator follow hello world t...,4
2,jshint throw possible strict violation line tr...,3
3,programmatically bound column invisible try da...,9
4,edittext get focus soft keyboard android home ...,4


In [19]:
X, y = df['cleaned_text'].values, df['Tag_Number_final'].values
X_train, X_t, y_train, y_t = train_test_split(X, y, test_size = 0.20, random_state=42)

In [20]:
X_test, X_valid, y_test, y_valid = train_test_split(X_t, y_t, test_size = 0.50, random_state=42)

In [21]:
class CustomDataset(torch.utils.data.Dataset):
    """IMDB dataset."""

    def __init__(self, X, y):
        self.X = np.array(X)
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        text = self.X[idx]
        labels = self.y[idx]
        sample = (text, labels)
        
        return sample

In [22]:
trainset = CustomDataset(X_train,y_train)
validset = CustomDataset(X_valid,y_valid)
testset = CustomDataset(X_test,y_test)

In [23]:
def create_vocab(dataset, min_freq):
  counter = Counter()
  for (text, _) in dataset:
    counter.update(str(text).split())
  my_vocab = vocab(counter, min_freq=min_freq)
  my_vocab.insert_token('<unk>', 0)
  my_vocab.set_default_index(0)
  return my_vocab

In [24]:
multiclass_vocab = create_vocab(trainset, min_freq = 5)

In [25]:
len(multiclass_vocab)

84078

In [26]:
#multiclass_vocab.get_itos()

In [27]:
# Creating a lambda function objects that will be used to get the indices of words from vocab
text_pipeline = lambda x: [multiclass_vocab[token] for token in str(x).split()]
label_pipeline = lambda x: int(x)

In [28]:
'''
We know that input to the embedding layers are indices of words from the vocab.
The collate_batch() accepts batch of data and gets the indices of text from vocab and returns the same
We will include this collate_batch() in collat_fn attribute of DataLoader.
So it will create a batch of data containing indices of words and corresponding labels.
But for EmbeddingBag we need one more extra parameter, that is offset.
offsets determines the starting index position of each bag (sequence) in input.
'''
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_text, _label) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return text_list, label_list, offsets

In [29]:
batch_size=2
check_loader= torch.utils.data.DataLoader(dataset=trainset,
                                        batch_size=batch_size,
                                        shuffle=True,
                                        collate_fn=collate_batch,
                                        num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [30]:
for text, label, offsets in check_loader:
  print(label, text, offsets)
  break

tensor([6, 0]) tensor([    0, 13426,   364,   845,    22,    38,   849, 74397,   851,   955,
           77,     0,  1596,    30,   104,   851,   955,    77,     0,  1596,
           30,   104,   851,   955,    77,     0,  1596,    30,   104,   851,
          955,    77,     0,  1596,    30,   104,   851,  2554,    76, 74397,
           78, 13195,  1596,    30,   104,   851,  1727,     0,    30,   104,
          851,  1727,     0,    30,   104,   851,     0,    22, 74397,   851,
          955,    77,     0,  1596,    30,   104,   851,   955,    77,     0,
         1596,    30,   104,   851,   955,    77,     0,  1596,    30,   104,
          851,   955,    77,     0,  1596,    30,   104,   851,  2554,    76,
        74397,    78,     0,  1596,    30,   104,   851,  1727,     0,    30,
          104,   851,  1727,     0,    30,   104,   336,   955,    77,     0,
         1596,    27,     0,   955,    77,     0,  1596,    27,     0,   955,
           77,     0,  1596,    27,     0,   955,

In [31]:
class MLPCustom(nn.Module):
  def __init__(self, embed_dim, vocab_size, h_sizes_list, d_prob_list, output_dim, non_linearity, batch_norm):

    super().__init__()

    self.output_dim = output_dim
    self.vocab_size = vocab_size
    self.embed_dim = embed_dim
    self.h_sizes_list = h_sizes_list
    self.d_prob_list = d_prob_list
    self.batch_norm = batch_norm
    
    self.non_linearity = non_linearity

    model_layers = []

    # embedding_layer
    self.embedding = nn.EmbeddingBag(self.vocab_size, self.embed_dim)

    input_dim = self.embed_dim
    # hidden layers, droput, non_linearity, batchnorm layers
    for k, hidden_size in enumerate(self.h_sizes_list):
      # hidden_layer
      model_layers.append(nn.Linear(input_dim, hidden_size))
      # Activation function
      model_layers.append(self.non_linearity)
      # Dropout Layer
      model_layers.append(nn.Dropout(p=self.d_prob_list[k]))
      # Batch_Norm Layer
      if self.batch_norm:
        model_layers.append(nn.BatchNorm1d(hidden_size, momentum = 0.9))
      input_dim = hidden_size

    # output layer  
    if len(self.h_sizes_list)>0:
        model_layers.append(nn.Linear(self.h_sizes_list[-1], self.output_dim))
    else:
        model_layers.append(nn.Linear(self.embed_dim, self.output_dim))

    self.module_list = nn.ModuleList(model_layers)
    

  def forward(self, x, offsets):
    out = self.embedding(x, offsets) # batchsize, embedding_dim
    for layer in self.module_list:
      out = layer(out)
    return out

    # Note : We do not need to apply softmax as we will be using nn.CrossEntropy Loss

In [32]:
def train(train_loader, loss_function, model, optimizer, grad_clipping, max_norm, log_batch, log_interval):

  # Training Loop 

  # initilalize variables as global
  # these counts will be updated every epoch
  global batch_ct_train

  # Initialize train_loss at the he start of the epoch
  running_train_loss = 0
  running_train_correct = 0
  
  # put the model in training mode

  model.train()
  # Iterate on batches from the dataset using train_loader
  for input_, targets, offsets in train_loader:
    
    # move inputs and outputs to GPUs
    input_ = input_.to(device)
    targets = targets.to(device)
    offsets = offsets.to(device)


    # Step 1: Forward Pass: Compute model's predictions 
    output = model(input_, offsets)
    
    # Step 2: Compute loss
    loss = loss_function(output, targets)

    # Correct prediction
    y_pred = torch.argmax(output, dim = 1)
    correct = torch.sum(y_pred == targets)

    batch_ct_train += 1

    # Step 3: Backward pass -Compute the gradients
    optimizer.zero_grad()
    loss.backward()

    # Gradient Clipping
    if grad_clipping:
      nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_norm, norm_type=2)

    # Step 4: Update the parameters
    optimizer.step()
          
    # Add train loss of a batch 
    running_train_loss += loss.item()

    # Add Corect counts of a batch
    running_train_correct += correct

    # log batch loss and accuracy
    if log_batch:
      if ((batch_ct_train + 1) % log_interval) == 0:
        wandb.log({f"Train Batch Loss  :": loss})
        wandb.log({f"Train Batch Acc :": correct/len(targets)})

  
  # Calculate mean train loss for the whole dataset for a particular epoch
  train_loss = running_train_loss/len(train_loader)

  # Calculate accuracy for the whole dataset for a particular epoch
  train_acc = running_train_correct/len(train_loader.dataset)
  

  return train_loss, train_acc

In [33]:
def validate(valid_loader, loss_function, model, log_batch, log_interval):

  # initilalize variables as global
  # these counts will be updated every epoch
  global batch_ct_valid

  # Validation/Test loop
  # Initialize valid_loss at the he strat of the epoch
  running_val_loss = 0
  running_val_correct = 0

  # put the model in evaluation mode
  model.eval()

  with torch.no_grad():
    for input_, targets, offsets in valid_loader:

      # move inputs and outputs to GPUs
      input_ = input_.to(device)
      targets = targets.to(device)
      offsets = offsets.to(device)

      # Step 1: Forward Pass: Compute model's predictions 
      output = model(input_, offsets)

      # Step 2: Compute loss
      loss = loss_function(output, targets)

      # Correct Predictions
      y_pred = torch.argmax(output, dim = 1)
      correct = torch.sum(y_pred == targets)

      batch_ct_valid += 1

      # Add val loss of a batch 
      running_val_loss += loss.item()

      # Add correct count for each batch
      running_val_correct += correct

      # log batch loss and accuracy
      if log_batch:
        if ((batch_ct_valid + 1) % log_interval) == 0:
          wandb.log({f"Valid Batch Loss  :": loss})
          wandb.log({f"Valid Batch Accuracy :": correct/len(targets)})

    # Calculate mean val loss for the whole dataset for a particular epoch
    val_loss = running_val_loss/len(valid_loader)

    # Calculate accuracy for the whole dataset for a particular epoch
    val_acc = running_val_correct/len(valid_loader.dataset)

    # scheduler step
    # scheduler.step(valid_loss)
    # scheduler.step()
    
  return val_loss, val_acc

In [34]:
def train_loop(train_loader, valid_loader, model, optimizer, loss_function, epochs, device, patience, early_stopping,
               file_model, save_best_model):
    
  """ 
  Function for training the model and plotting the graph for train & validation loss vs epoch.
  Input: iterator for train dataset, initial weights and bias, epochs, learning rate, batch size.
  Output: final weights, bias and train loss and validation loss for each epoch.
  """

  # Create lists to store train and val loss at each epoch
  train_loss_history = []
  valid_loss_history = []
  train_acc_history = []
  valid_acc_history = []

  # initialize variables for early stopping

  delta = 0
  best_score = None
  valid_loss_min = np.Inf
  counter_early_stop=0
  early_stop=False

  # Iterate for the given number of epochs
  # Step 5: Repeat steps 1 - 4

  for epoch in range(epochs):

    t0 = datetime.now()

    # Get train loss and accuracy for one epoch
    train_loss, train_acc = train(train_loader, loss_function, model, optimizer, 
                                  wandb.config.GRAD_CLIPPING, wandb.config.MAX_NORM,
                                  wandb.config.LOG_BATCH, wandb.config.LOG_INTERVAL)
    valid_loss, valid_acc   = validate(valid_loader, loss_function, model, 
                                       wandb.config.LOG_BATCH, wandb.config.LOG_INTERVAL)

    dt = datetime.now() - t0

    # Save history of the Losses and accuracy
    train_loss_history.append(train_loss)
    train_acc_history.append(train_acc)

    valid_loss_history.append(valid_loss)
    valid_acc_history.append(valid_acc)

    # Log the train and valid loss to wandb
    wandb.log({f"Train Loss :": train_loss, "epoch": epoch})
    wandb.log({f"Train Acc :": train_acc, "epoch": epoch})

    wandb.log({f"Valid Loss :": valid_loss, "epoch": epoch})
    wandb.log({f"Valid Acc :": valid_acc, "epoch": epoch})

    if early_stopping:
      score = -valid_loss
      if best_score is None:
        best_score=score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving Model...')
        torch.save(model.state_dict(), file_model)
        valid_loss_min = valid_loss

      elif score < best_score + delta:
        counter_early_stop += 1
        print(f'Early stoping counter: {counter_early_stop} out of {patience}')
        if counter_early_stop > patience:
          early_stop = True


      else:
        best_score = score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), file_model)
        counter_early_stop=0
        valid_loss_min = valid_loss

      if early_stop:
        print('Early Stopping')
        break

    elif save_best_model:

      score = -valid_loss
      if best_score is None:
        best_score=score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving Model...')
        torch.save(model.state_dict(), file_model)
        valid_loss_min = valid_loss

      elif score < best_score + delta:
        print(f'Validation loss has not decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Not Saving Model...')
      
      else:
        best_score = score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), file_model)
        valid_loss_min = valid_loss
        
    else:
        torch.save(model.state_dict(), file_model)
    
    # Print the train loss and accuracy for given number of epochs, batch size and number of samples
    print(f'Epoch : {epoch+1} / {epochs}')
    print(f'Time to complete {epoch+1} is {dt}')
    # print(f'Learning rate: {scheduler._last_lr[0]}')
    print(f'Train Loss: {train_loss : .4f} | Train Accuracy: {train_acc * 100 : .4f}%')
    print(f'Valid Loss: {valid_loss : .4f} | Valid Accuracy: {valid_acc * 100 : .4f}%')
    print()
    torch.cuda.empty_cache()

  return train_loss_history, train_acc_history, valid_loss_history, valid_acc_history

In [35]:
def get_acc_pred(data_loader, model, device):
    
  """ 
  Function to get predictions and accuracy for a given data using estimated model
  Input: Data iterator, Final estimated weoights, bias
  Output: Prections and Accuracy for given dataset
  """

  # Array to store predicted labels
  predictions = torch.Tensor() # empty tensor
  predictions = predictions.to(device) # move predictions to GPU

  # Array to store actual labels
  y = torch.Tensor() # empty tensor
  y = y.to(device)

  # put the model in evaluation mode
  model.eval()
  
  # Iterate over batches from data iterator
  with torch.no_grad():
    for input_, targets, offsets in data_loader:
      
      # move inputs and outputs to GPUs
      
      input_ = input_.to(device)
      targets = targets.to(device)
      offsets = offsets.to(device)
      
      # Calculated the predicted labels
      output = model(input_, offsets)

      # Choose the label with maximum probability
      prediction = torch.argmax(output, dim = 1)

      # Add the predicted labels to the array
      predictions = torch.cat((predictions, prediction)) 

      # Add the actual labels to the array
      y = torch.cat((y, targets)) 

  # Check for complete dataset if actual and predicted labels are same or not
  # Calculate accuracy
  acc = (predictions == y).float().mean()

  # Return tuple containing predictions and accuracy
  return predictions, acc  

In [36]:
hyperparameters = SimpleNamespace(
    EMBED_DIM = 300,
    VOCAB_SIZE = len(multiclass_vocab),
    OUTPUT_DIM = 10,
    HIDDEN_SIZES_LIST = [200, 100], # 100 layers of size 200  [200]*100
    DPROB_LIST = [0.5, 0.5],
    NON_LINEARITY= nn.ReLU(),
    BATCH_NORM = False,
    EPOCHS = 20,
    
    BATCH_SIZE = 256,
    LEARNING_RATE = 0.2,
    DATASET="MULTICLASS",
    ARCHITECTUREe="Embed_2_hidden_layers",
    LOG_INTERVAL = 25,
    LOG_BATCH = True,
    FILE_MODEL = model_folder/'multiclass.pt',
    GRAD_CLIPPING = False,
    MAX_NORM = 0,
    MOMENTUM = 0,
    PATIENCE = 5,
    EARLY_STOPPING = True,
    # SCHEDULER_FACTOR = 0,
    # SCHEDULER_PATIENCE = 0,
    WEIGHT_DECAY = 0,
    SAVE_BEST_MODEL = False,
    DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    )

In [37]:
# Initialize a new project
import random
wandb.init(name = 'Embed_2_hidden_layers', project = 'NLP_MLP_Template', config = hyperparameters)

wandb: Currently logged in as: hamshaks1. Use `wandb login --relogin` to force relogin


In [38]:
wandb.config = hyperparameters
wandb.config

namespace(ARCHITECTUREe='Embed_2_hidden_layers', BATCH_NORM=False, BATCH_SIZE=256, DATASET='MULTICLASS', DEVICE=device(type='cpu'), DPROB_LIST=[0.5, 0.5], EARLY_STOPPING=True, EMBED_DIM=300, EPOCHS=20, FILE_MODEL=PosixPath('/content/drive/MyDrive/NLP_Fall22/HW5/Models/multiclass.pt'), GRAD_CLIPPING=False, HIDDEN_SIZES_LIST=[200, 100], LEARNING_RATE=0.2, LOG_BATCH=True, LOG_INTERVAL=25, MAX_NORM=0, MOMENTUM=0, NON_LINEARITY=ReLU(), OUTPUT_DIM=10, PATIENCE=5, SAVE_BEST_MODEL=False, VOCAB_SIZE=84078, WEIGHT_DECAY=0)

In [39]:
# Fix seed value
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Data Loader
train_loader = torch.utils.data.DataLoader(trainset, batch_size=wandb.config.BATCH_SIZE, shuffle = True, 
                                           collate_fn=collate_batch, num_workers = 4)
valid_loader = torch.utils.data.DataLoader(validset, batch_size=wandb.config.BATCH_SIZE, shuffle = False, 
                                           collate_fn=collate_batch,  num_workers = 4)
test_loader = torch.utils.data.DataLoader(testset, batch_size=wandb.config.BATCH_SIZE,   shuffle = False, 
                                          collate_fn=collate_batch,  num_workers = 4)

# cross entropy loss function
loss_function = nn.CrossEntropyLoss()

# model 
model_multiclass = MLPCustom(wandb.config.EMBED_DIM, 
                       wandb.config.VOCAB_SIZE, 
                       wandb.config.HIDDEN_SIZES_LIST, 
                       wandb.config.DPROB_LIST,
                       wandb.config.OUTPUT_DIM, 
                       wandb.config.NON_LINEARITY,
                       wandb.config.BATCH_NORM)

model_multiclass.to(wandb.config.DEVICE)

def init_weights(m):
  if type(m) == nn.Linear:
      torch.nn.init.kaiming_normal_(m.weight)
      torch.nn.init.zeros_(m.bias)
        
# apply initialization recursively  to all modules
model_multiclass.apply(init_weights)

# Intialize stochiastic gradient descent optimizer
optimizer = torch.optim.SGD(model_multiclass.parameters(), 
                             lr = wandb.config.LEARNING_RATE, 
                             weight_decay=wandb.config.WEIGHT_DECAY)

wandb.config.OPTIMIZER = optimizer

# scheduler = ReduceLROnPlateau(optimizer, mode='min', factor= wandb.config.scheduler_factor, 
#                              patience=wandb.config.scheduler_patience, verbose=True)

#scheduler = StepLR(optimizer, gamma=0.4,step_size=1, verbose=True)

In [40]:
wandb.config

namespace(ARCHITECTUREe='Embed_2_hidden_layers', BATCH_NORM=False, BATCH_SIZE=256, DATASET='MULTICLASS', DEVICE=device(type='cpu'), DPROB_LIST=[0.5, 0.5], EARLY_STOPPING=True, EMBED_DIM=300, EPOCHS=20, FILE_MODEL=PosixPath('/content/drive/MyDrive/NLP_Fall22/HW5/Models/multiclass.pt'), GRAD_CLIPPING=False, HIDDEN_SIZES_LIST=[200, 100], LEARNING_RATE=0.2, LOG_BATCH=True, LOG_INTERVAL=25, MAX_NORM=0, MOMENTUM=0, NON_LINEARITY=ReLU(), OPTIMIZER=SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.2
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
), OUTPUT_DIM=10, PATIENCE=5, SAVE_BEST_MODEL=False, VOCAB_SIZE=84078, WEIGHT_DECAY=0)

In [41]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
for input_, targets, offsets in train_loader:
  
  # move inputs and outputs to GPUs
  input_ = input_.to(device)
  targets = targets.to(device)
  offsets = offsets.to(device)
  model_multiclass.eval()
  # Forward pass
  output = model_multiclass(input_, offsets)
  loss = loss_function(output, targets)
  print(f'Actual loss: {loss}')
  break

print(f'Expected Theoretical loss: {np.log(2)}')

Actual loss: 2.380561351776123
Expected Theoretical loss: 0.6931471805599453


In [42]:
wandb.watch(model_multiclass, log = 'all', log_freq=25, log_graph=True)

wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


In [43]:
# See live graphs in the notebook.
#%%wandb 
batch_ct_train, batch_ct_valid = 0, 0
train_loss_history, train_acc_history, valid_loss_history, valid_acc_history = train_loop(train_loader, 
                                                                                          valid_loader, 
                                                                                          model_multiclass, 
                                                                                          optimizer,
                                                                                          loss_function, 
                                                                                          wandb.config.EPOCHS, 
                                                                                          wandb.config.DEVICE,
                                                                                          wandb.config.PATIENCE, 
                                                                                          wandb.config.EARLY_STOPPING,
                                                                                          wandb.config.FILE_MODEL,
                                                                                          wandb.config.SAVE_BEST_MODEL)

Validation loss has decreased (inf --> 0.832393). Saving Model...
Epoch : 1 / 20
Time to complete 1 is 0:01:21.320497
Train Loss:  1.4000 | Train Accuracy:  53.4957%
Valid Loss:  0.8324 | Valid Accuracy:  73.9305%

Validation loss has decreased (0.832393 --> 0.711477). Saving model...
Epoch : 2 / 20
Time to complete 2 is 0:01:22.717359
Train Loss:  0.9576 | Train Accuracy:  69.9613%
Valid Loss:  0.7115 | Valid Accuracy:  77.8695%

Validation loss has decreased (0.711477 --> 0.658536). Saving model...
Epoch : 3 / 20
Time to complete 3 is 0:01:20.512033
Train Loss:  0.8489 | Train Accuracy:  73.7601%
Valid Loss:  0.6585 | Valid Accuracy:  79.7596%

Validation loss has decreased (0.658536 --> 0.621304). Saving model...
Epoch : 4 / 20
Time to complete 4 is 0:01:22.580376
Train Loss:  0.7868 | Train Accuracy:  75.5788%
Valid Loss:  0.6213 | Valid Accuracy:  80.6491%

Validation loss has decreased (0.621304 --> 0.610166). Saving model...
Epoch : 5 / 20
Time to complete 5 is 0:01:22.406436
Tr

In [44]:
device

device(type='cpu')

In [45]:
model_nn = MLPCustom(wandb.config.EMBED_DIM, 
                       wandb.config.VOCAB_SIZE, 
                       wandb.config.HIDDEN_SIZES_LIST, 
                       wandb.config.DPROB_LIST,
                       wandb.config.OUTPUT_DIM, 
                       wandb.config.NON_LINEARITY,
                       wandb.config.BATCH_NORM)
model_nn.to(device)
model_nn.load_state_dict(torch.load(wandb.config.FILE_MODEL))

<All keys matched successfully>

In [46]:
# Get the prediction and accuracy for the test dataseta
predictions_test, acc_test = get_acc_pred(test_loader, model_nn, device)
predictions_train, acc_train = get_acc_pred(train_loader, model_nn, device)
predictions_valid, acc_valid = get_acc_pred(valid_loader, model_nn, device)

In [47]:
# Print Test Accuracy
print('Test accuracy', acc_test * 100)
print('Train accuracy', acc_train * 100)
print('Valid accuracy', acc_valid * 100)

Test accuracy tensor(84.8952)
Train accuracy tensor(87.1438)
Valid accuracy tensor(85.4670)


In [48]:
wandb.log({'Best_test_Acc': acc_test})
wandb.log({'Best_train_Acc': acc_train})
wandb.log({'Best_valid_Acc': acc_valid})

In [49]:
# Get an array containing actual labels
testing_labels = testset.y

In [50]:
np.unique(testing_labels)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [51]:
# Log a confusion matrix to W&B
wandb.log({"conf_mat" : wandb.plot.confusion_matrix(
                        probs = None,
                        y_true = testing_labels,
                        preds = predictions_test.to('cpu').numpy(),
                        class_names =['c#', 'java', 'php', 'javascript', 'android', 'jquery', 'c++', 'python', 'iphone', 'asp.net'])})

In [52]:
wandb.finish()

Best_test_Acc,▁
Best_train_Acc,▁
Best_valid_Acc,▁
Train Acc :,▁▅▆▆▆▇▇▇▇▇▇▇████████
Train Batch Acc :,▁▄▄▄▅▅▆▅▆▆▇▆▆▇▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇█▇▇▇▇
Train Batch Loss :,█▅▅▄▄▄▃▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▂▂▂
Train Loss :,█▅▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁
Valid Acc :,▁▃▅▅▅▆▆▆▆▇▇▇▇▇██████
Valid Batch Accuracy :,▁▂▃▂▄▅▅▆▄▄▅▄▄▅▆▆▆▅▄▄▅▅▆▅▇▅▄█▆▅▅▆▆▆▅▇▇▇█▆
Valid Batch Loss :,█▇▆▇▆▄▄▄▆▅▄▃▄▃▃▂▃▄▄▅▃▄▃▄▂▃▅▂▃▃▄▄▂▃▂▂▂▂▁▃
Valid Loss :,█▆▅▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁
